In [22]:
from mip import *

In [23]:
def readFile(filename):
    File = open(filename, 'r')
    lines = File.readlines()

    # número de variáveis
    n = int(lines[0].split()[0])
    # número de restrições
    r = int(lines[0].split()[1])

    File.close()

    m = Model(sense=MAXIMIZE, solver_name=CBC)
    x = [m.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0,
                    name=f"x_{i}") for i in range(n)]

    obj = [float(i) for i in lines[1].split()]
    m.objective = xsum(obj[i]*x[i] for i in range(n))


    # coeficientes
    for c in range (2, r+1):
        coeficientes = []
        for i in range(n + 1):
            coe = float(lines[c].split()[i])
            coeficientes.append(coe)
            
    m += xsum(coeficientes[j]*x[j] for j in range(n)) <= coeficientes[-1]

    m.verbose = 0

    return m

In [24]:
# resolve o modelo e mostra os valores das variáveis
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  print(f"Solution value  = {model.objective_value:.2f}\n")

  print("Solution:")
  for v in model.vars:
      print(f"{v.name} = {v.x:.2f}")


# salva modelo em arquivo lp, e mostra o conteúdo
def save(model, filename):
  model.write(filename) # salva modelo em arquivo
  with open(filename, "r") as f: # lê e exibe conteúdo do arquivo
    print(f.read())

In [25]:
class No:
    def __init__ (self, modelo):
        self.modelo = modelo
        self.modelo.verbose = 0
        self.modelo.optimize()
        self.Z = self.modelo.objective_value


In [26]:
# poda por integralidade, analisa todas as variáveis do modelo
def poda_integralidade (modelo):
    # Se alguma variável não for inteira, o nó é descartado
    for v in modelo.vars:
        if v.x % 1 != 0:
            return False
    return True

# poda por limitante, analisa o valor da função objetivo
def poda_limitante (lim_inf, Z):
    # Se o valor da função objetivo for menor ou igual ao limitante inferior,
    # qualquer nó abaixo incluindo o próprio nó é descartado,
    # pois já há uma solução melhor encontrada
    if Z <= lim_inf:
        return True
    else:
        return False

# poda por inviabilidade, analisa o status do modelo
def poda_inviabilidade (modelo):
    # Inviável
    if modelo.status == OptimizationStatus.INFEASIBLE:
        return True
    # Não encontrou solução
    elif modelo.status == OptimizationStatus.NO_SOLUTION_FOUND:
        return True
    
    return False

In [27]:
def print_no(filho: Model):
    print("Status = ", filho.status)
    print(f"Solution value  = {filho.objective_value:.2f}\n")

    print("Solution:")
    for v in filho.vars:
        print(f"{v.name} = {v.x:.2f}")

In [28]:
# Gera os dois nós filhos do algoritmo a partir da variável mais próxima de 0.5
def passar_no (pai):
    dif_abs = 1
    for var in pai.modelo.vars:
        if abs(0.5 - var.x) <= dif_abs:
            dif_abs = abs(0.5 - var.x)
            escolhida = var.name
    
    # cria os modelos filhos
    m1 = pai.modelo.copy()
    m2 = pai.modelo.copy()

    # adiciona restrição para a variável escolhida
    m1 += m1.vars[escolhida] == 0
    m2 += m2.vars[escolhida] == 1

    # cria os nós filhos
    filho1 = No(m1)
    filho2 = No(m2)

    return filho1, filho2


In [29]:
class Branch_and_Bound:
    def __init__ (self, no_modelo):
        self.raiz = no_modelo

    def solve(self):
        lim_inf = float('-inf')
        fronteira = [self.raiz]
        solucao = None

        while fronteira:
            no = fronteira.pop(0)
            if poda_integralidade(no.modelo):
                if no.Z > lim_inf:
                    lim_inf = no.Z
                    solucao = no.modelo
            
            if poda_inviabilidade(no.modelo):
                continue

            if poda_limitante(lim_inf, no.Z):
                continue

            filho1, filho2 = passar_no(no)
            fronteira.append(filho1)
            fronteira.append(filho2)

        print_no(solucao)

In [30]:
teste1 = No(readFile('/testes/teste1.txt'))

In [31]:
BnB = Branch_and_Bound(teste1)

In [32]:
BnB.solve()


Status =  OptimizationStatus.OPTIMAL
Solution value  = 53.00

Solution:
x_0 = 1.00
x_1 = 1.00
x_2 = 1.00
x_3 = 1.00
x_4 = 1.00
x_5 = 1.00
x_6 = 1.00
